In [1]:
## CUTRUN DE and assign to nearest genes

library(pacman)
p_load(data.table, dplyr, ggplot2, viridis, magrittr, VennDiagram, ggpubr, limma, edgeR, tidyr, GenomicRanges, RColorBrewer, pheatmap, Seurat, fgsea, GSEABase, limma, TxDb.Hsapiens.UCSC.hg38.knownGene, org.Hs.eg.db)

cell_list <- c("N", "CM", "EM", "EMRA")
hd_list <- paste0("HD", c(1:3, 5:7))
expr_list = c("Input", "Product", "Stim1", "Stim2", "Stim3")
cell_comp_list = c("N_CM", "N_EM", "N_EMRA", "CM_EM", "CM_EMRA", "EM_EMRA")
hist_list <- c("H3K27me3", "H3K4me2")

in_path <- "CART_CUTRUN_Project/results/RNAseq/process/RSEM/"
out_path <- "CART_CUTRUN_Project/results/RNAseq/analysis/RSEM/"
fig_path <- "CART_CUTRUN_Project/results/paper_figure/"
inPath = "CART_CUTRUN_Project/results/CUTANDRUN/process/"
outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"


In [ ]:
## PCA for RNA and CUTRUN
load(file = paste0(outPath, "/RData/countMat_designInfo_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData")) ## 'cell_comp_list''cell_list''countMat''designInfo''expr_list''fig_path''hd_list''hist_list''in_path''inPath''out_path''outPath'
load(file = paste0(outPath, "/RData/masterPeak_peakAnno_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = "CART_CUTRUN_Project/results/RNAseq/analysis/RSEM/RData/HD1-3_5-7_normLimma_perExprCondition.RData") ## data, selectR, dataS, voomDDS, results, 

geneID = fread(paste0("CART_CUTRUN_Project/results/RNAseq/process/RSEM/", "RNA_CD8_N_Input_HD1.genes.results"))$gene_id
data = data.frame(data)
select_exp = "Input"

rna_m = data[, (colnames(data) %>% stringr::str_detect(select_exp)) & !(colnames(data) %>% stringr::str_detect("EMRA"))]
k27_m = countMat[["H3K27me3"]][, (colnames(countMat[["H3K27me3"]]) %>% stringr::str_detect(select_exp)) & !(colnames(countMat[["H3K27me3"]]) %>% stringr::str_detect("EMRA"))]
k4_m = countMat[["H3K4me2"]][, (colnames(countMat[["H3K4me2"]]) %>% stringr::str_detect(select_exp)) & !(colnames(countMat[["H3K4me2"]]) %>% stringr::str_detect("EMRA"))]

rownames(rna_m) = geneID
rownames(k27_m) = mPeak[["H3K27me3"]] %>% data.frame %>% mutate(region = paste0(seqnames, ":", start, "-", end)) %$% region
rownames(k4_m) = mPeak[["H3K4me2"]] %>% data.frame %>% mutate(region = paste0(seqnames, ":", start, "-", end)) %$% region

dim(rna_m)
dim(k27_m)
dim(k4_m)
head(rna_m)
head(k27_m)
head(k4_m)

rna_voom = voomDDS$E[, colnames(voomDDS$E) %>% stringr::str_detect(select_exp)]
load(file = paste0(outPath, "/RData/results_histList_hd1-7_adjustPeak_noChrXYM.RData"))
k27_voom = voomDDS[["H3K27me3"]]$E[, colnames(voomDDS[["H3K27me3"]]$E) %>% stringr::str_detect(select_exp)]
k4_voom = voomDDS[["H3K4me2"]]$E[, colnames(voomDDS[["H3K4me2"]]$E) %>% stringr::str_detect(select_exp)]
rownames(rna_voom) = 1:nrow(rna_voom)
rownames(k27_voom) = paste0("K27me3_", 1:nrow(k27_voom))
rownames(k4_voom) = paste0("K4me2_", 1:nrow(k4_voom))

dim(rna_voom)
dim(k27_voom)
dim(k4_voom)

In [ ]:
rna_obj = CreateSeuratObject(counts = rna_m, project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 2000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 2000) %>% 
ScaleData()
rna_obj = RunPCA(rna_obj, features = VariableFeatures(object = rna_obj), npcs = 15)

# k27_obj = CreateSeuratObject(counts = k27_m, project = "cart", min.cells = 0, min.features = 10) %>%  
# NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
# FindVariableFeatures(., selection.method = "vst", nfeatures = 1000) %>% 
# ScaleData()
# # FindVariableFeatures(., selection.method = "vst", nfeatures = round(nrow(k27_m)/10*8)) %>% 

# k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 10)

# k4_obj = CreateSeuratObject(counts = k4_m, project = "cart", min.cells = 0, min.features = 50) %>%  
# NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
# FindVariableFeatures(., selection.method = "vst", nfeatures = 1000) %>%  #
# ScaleData()
# k4_obj = RunPCA(k4_obj, features = VariableFeatures(object = k4_obj), npcs = 10)


# cr_obj = CreateSeuratObject(counts = rbind(k27_m, k4_m), project = "cart", min.cells = 0, min.features = 50) %>%  
# NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
# FindVariableFeatures(., selection.method = "vst", nfeatures = 1000 ) %>%  #round((nrow(k27_m) + nrow(k4_m))/10*8)
# ScaleData()
# cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 10)



In [ ]:
options(repr.plot.width=15, repr.plot.height=5)
# DimPlot(rna_obj, reduction = "pca")
ct_list = c("N", "CM", "EM")
p1 = Embeddings(rna_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(celltype = factor(rep(ct_list, each = 6), levels = ct_list)) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = celltype)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0, end = 0.75) +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("RNA-seq") +
rremove("legend.title")

p2 = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(celltype = factor(rep(ct_list, each = 4), levels = ct_list)) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = celltype)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0, end = 0.75) +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
rremove("legend.title")

p3 = Embeddings(k4_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(celltype = factor(rep(ct_list, each = 4), levels = ct_list)) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = celltype)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0, end = 0.75) +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
rremove("legend.title")

p4 = Embeddings(cr_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(celltype = factor(rep(ct_list, each = 4), levels = ct_list)) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = celltype)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0, end = 0.75) +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3+H3K4me2") +
rremove("legend.title")

ggarrange(p1, p2, p3, p4, nrow = 1, ncol = 4, common.legend = TRUE)
# ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig1/PCA_seurat_procedure_N_CM_EM.pdf", width = 20, height = 6)

In [5]:
library(pacman)
p_load(data.table, dplyr, ggplot2, viridis, magrittr, VennDiagram, ggpubr, limma, edgeR, tidyr, GenomicRanges, RColorBrewer, pheatmap, Seurat, fgsea, GSEABase, limma, TxDb.Hsapiens.UCSC.hg38.knownGene, org.Hs.eg.db, ggrepel)
fig_path <- "CART_CUTRUN_Project/results/paper_figure_paper1/"

outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"

x_thres = 4.5
y_thres = 2.5

In [30]:
cT = "N"
cr_rna_match = readRDS(file = paste0(outPath, "/RDS/cr_rna_match_", cT, ".rds"))
p_n = cr_rna_match %>% ggplot(aes(x = H3K4me2, y = H3K27me3, color = RNA)) +
        geom_point(size = 1) +
        theme_bw(base_size = 30) +
        xlab("H3K4me2: log2(scaled count + 1)") +
        ylab("H3K27me3: log2(scaled count + 1)") +
        scale_color_viridis() +
        geom_vline(xintercept = x_thres, linetype = "dashed", color = 'grey10', size = 1.5) +
        geom_hline(yintercept = y_thres, linetype = "dashed", color = "grey10", size = 1.5) +
         labs(color = "RNA: log2(expected count + 1)") +  guides(color = guide_colourbar(barwidth = 10, barheight = 1))+
        ggtitle("Naive")

cT = "CM"
cr_rna_match = readRDS(file = paste0(outPath, "/RDS/cr_rna_match_", cT, ".rds"))
p_cm = cr_rna_match %>% ggplot(aes(x = H3K4me2, y = H3K27me3, color = RNA)) +
        geom_point(size = 1) +
        theme_bw(base_size = 30) +
        xlab("H3K4me2: log2(scaled count + 1)") +
        ylab("H3K27me3: log2(scaled count + 1)") +
        scale_color_viridis() +
        geom_vline(xintercept = x_thres, linetype = "dashed", color = 'grey10', size = 1.5) +
        geom_hline(yintercept = y_thres, linetype = "dashed", color = "grey10", size = 1.5) +
         labs(color = "RNA: log2(expected count + 1)") +  guides(color = guide_colourbar(barwidth = 10, barheight = 1))+
        ggtitle("CM")

cT = "EM"
cr_rna_match = readRDS(file = paste0(outPath, "/RDS/cr_rna_match_", cT, ".rds"))
p_em = cr_rna_match %>% ggplot(aes(x = H3K4me2, y = H3K27me3, color = RNA)) +
        geom_point(size = 1) +
        theme_bw(base_size = 30) +
        xlab("H3K4me2: log2(scaled count + 1)") +
        ylab("H3K27me3: log2(scaled count + 1)") +
        scale_color_viridis() +
        geom_vline(xintercept = x_thres, linetype = "dashed", color = 'grey10', size = 1.5) +
        geom_hline(yintercept = y_thres, linetype = "dashed", color = "grey10", size = 1.5) +
        labs(color = "RNA: log2(expected count + 1)") +  guides(color = guide_colourbar(barwidth = 10, barheight = 1))+
        ggtitle("EM") 


In [ ]:
options(repr.plot.width=25, repr.plot.height=8)
ggarrange(p_n, p_cm, p_em, nrow = 1, common.legend = TRUE) 
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig2/K27_vs_K4_colorbyRNA_N_CM_EM.pdf", width = 21, height = 9)

In [14]:
cT = "N"
cr_rna_match = readRDS(file = paste0(outPath, "/RDS/cr_rna_match_", cT, ".rds"))
p_n = cr_rna_match %>% ggplot(aes(x = H3K4me2, y = H3K27me3, color = RNA)) +
        geom_point(size = 1, shape = 19, alpha = 0.2) +
        theme_bw(base_size = 30) +
        xlab("H3K4me2: log2(scaled count + 1)") +
        ylab("H3K27me3: log2(scaled count + 1)") +
        scale_color_viridis() +
        geom_vline(xintercept = x_thres, linetype = "dashed", color = 'grey10', size = 1.5) +
        geom_hline(yintercept = y_thres, linetype = "dashed", color = "grey10", size = 1.5) +
         labs(color = "RNA: log2(expected count + 1)") +   guides(color = guide_colourbar(barwidth = 15, barheight = 1))+
        ggtitle("Naive")

cT = "CM"
cr_rna_match = readRDS(file = paste0(outPath, "/RDS/cr_rna_match_", cT, ".rds"))
p_cm = cr_rna_match %>% ggplot(aes(x = H3K4me2, y = H3K27me3, color = RNA)) +
        geom_point(size = 1, shape = 19, alpha = 0.2) +
        theme_bw(base_size = 30) +
        xlab("H3K4me2: log2(scaled count + 1)") +
        ylab("H3K27me3: log2(scaled count + 1)") +
        scale_color_viridis() +
        geom_vline(xintercept = x_thres, linetype = "dashed", color = 'grey10', size = 1.5) +
        geom_hline(yintercept = y_thres, linetype = "dashed", color = "grey10", size = 1.5) +
        labs(color = "RNA: log2(expected count + 1)") +  guides(color = guide_colourbar(barwidth = 15, barheight = 1))+
        ggtitle("CM")

cT = "EM"
cr_rna_match = readRDS(file = paste0(outPath, "/RDS/cr_rna_match_", cT, ".rds"))
p_em = cr_rna_match %>% ggplot(aes(x = H3K4me2, y = H3K27me3, color = RNA)) +
        geom_point(size = 1, shape = 19, alpha = 0.2) +
        theme_bw(base_size = 30) +
        xlab("H3K4me2: log2(scaled count + 1)") +
        ylab("H3K27me3: log2(scaled count + 1)") +
        scale_color_viridis() +
        geom_vline(xintercept = x_thres, linetype = "dashed", color = 'grey10', size = 1.5) +
        geom_hline(yintercept = y_thres, linetype = "dashed", color = "grey10", size = 1.5) +
        labs(color = "RNA: log2(expected count + 1)") +  guides(color = guide_colourbar(barwidth = 15, barheight = 1)) + 
        ggtitle("EM") 


In [ ]:
options(repr.plot.width=25, repr.plot.height=8)
ggarrange(p_n, p_cm, p_em, nrow = 1, common.legend = TRUE) 
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig2/K27_vs_K4_colorbyRNA_N_CM_EM_opacity0.2.pdf", width = 21, height = 9)

In [ ]:
## Bivalent genes
## CUTRUN DE and assign to nearest genes

library(pacman)
p_load(data.table, dplyr, ggplot2, viridis, magrittr, VennDiagram, ggpubr, limma, edgeR, tidyr, GenomicRanges, RColorBrewer, pheatmap, Seurat, fgsea, GSEABase, limma, TxDb.Hsapiens.UCSC.hg38.knownGene, org.Hs.eg.db)

cell_list <- c("N", "CM", "EM", "EMRA")
hd_list <- paste0("HD", c(1:3, 5:7))
expr_list = c("Input", "Product", "Stim1", "Stim2", "Stim3")
cell_comp_list = c("N_CM", "N_EM", "N_EMRA", "CM_EM", "CM_EMRA", "EM_EMRA")
hist_list <- c("H3K27me3", "H3K4me2")

in_path <- "CART_CUTRUN_Project/results/RNAseq/process/RSEM/"
out_path <- "CART_CUTRUN_Project/results/RNAseq/analysis/RSEM/"
fig_path <- "CART_CUTRUN_Project/results/paper_figure/"
inPath = "CART_CUTRUN_Project/results/CUTANDRUN/process/"
outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"

## Bivalent methylation marks

load(file = paste0(outPath, "/RData/masterPeak_peakAnno_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = paste0(outPath, "/RData/IDR_master_peak_list_chromVar_count_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = paste0(outPath, "/RData/countMat_designInfo_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = paste0(outPath, "/RData/results_histList_hd1-7_adjustPeak_noChrXYM.RData"))

names(peakAll)

In [ ]:

## overlap K4me2 and K27me3 ensuring at least 50bp overlap, count number of regions after merging
fig_path = "CART_CUTRUN_Project/results/paper_figure/fig1"
bi_path <- "CART_CUTRUN_Project/results/CUTANDRUN/analysis/BivalentGenes/"
gene_gr <- genes(TxDb.Hsapiens.UCSC.hg38.knownGene, columns = "gene_id")

get_nearest_gene <- function(gene_gr, region_gr){
    region_assign_gene <- distanceToNearest(region_gr, gene_gr, select = "all")
    # dist_tmp <- region_assign_gene %>% data.frame %$% distance
    # region_assign_gene <- region_assign_gene[dist_tmp < 1000]
    region_match_gene <- cbind(data.frame(region_gr)[region_assign_gene@from, ], data.frame(peak_id = region_assign_gene@from, gene_name = mapIds(org.Hs.eg.db, gene_gr$gene_id[region_assign_gene@to], "SYMBOL", "ENTREZID") %>% as.vector) )
    return(region_match_gene)

}
bi_num <- c()
for(cT in c("N", "CM", "EM", "EMRA")){
    for(pt in paste0("HD", 4:7)){
        bi_olp <- findOverlaps(peakAll[[paste0("H3K27me3_", cT, "_Input_", pt)]], peakAll[[paste0("H3K4me2_", cT, "_Input_", pt)]], minoverlap = 50)
        bi_peak.gr <- reduce(append(peakAll[[paste0("H3K27me3_", cT, "_Input_", pt)]][bi_olp@from], peakAll[[paste0("H3K4me2_", cT, "_Input_", pt)]][bi_olp@to]))
        bi_gene <- get_nearest_gene(gene_gr, bi_peak.gr)
        write.table(unique(bi_gene$gene_name), paste0(bi_path, "Bivalent_peaks_nearest_genes_", cT, "_", pt, ".csv"), quote = F, row.names = F, col.names = F, sep = ",")

        total_peak.gr <- append(peakAll[[paste0("H3K27me3_", cT, "_Input_", pt)]], peakAll[[paste0("H3K4me2_", cT, "_Input_", pt)]]) %>% reduce
        total_gene <- get_nearest_gene(gene_gr, total_peak.gr)

        bi_num <- data.frame(
            bi_peak_num = length(bi_peak.gr),
            bi_gene_num = length(unique(bi_gene$gene_name)),
            total_peak_num = length(total_peak.gr),
            total_gene_num = length(unique(total_gene$gene_name)),
            cT = cT,
            pt = pt
        ) %>% rbind(bi_num, .)
    }
}
bi_num$cT <- factor(bi_num$cT, levels = c("N", "CM", "EM", "EMRA"))

# p1 = bi_num %>% ggplot(aes(x = pt, y = bi_peak_num, fill = cT)) + 
# geom_bar(stat = "identity", position = position_dodge()) +
# scale_fill_viridis(discrete = TRUE) +
# theme_bw(base_size = 20) +
# xlab("") +
# ylab("# of Peak Regions") +
# rremove("legend.title")

# p2 = bi_num %>% mutate(bi_peak_prop = bi_peak_num/total_peak_num * 100) %>% ggplot(aes(x = pt, y = bi_peak_prop, fill = cT)) + 
# geom_bar(stat = "identity", position = position_dodge()) +
# scale_fill_viridis(discrete = TRUE) +
# theme_bw(base_size = 20) +
# xlab("") +
# ylab("% of Peak Regions") +
# rremove("legend.title")

# p3 = bi_num %>% ggplot(aes(x = pt, y = bi_gene_num, fill = cT)) + 
# geom_bar(stat = "identity", position = position_dodge()) +
# scale_fill_viridis(discrete = TRUE) +
# theme_bw(base_size = 20) +
# xlab("") +
# ylab("# of Genes") +
# rremove("legend.title")

# p4 = bi_num %>% mutate(bi_gene_prop = bi_gene_num/total_gene_num * 100) %>% ggplot(aes(x = pt, y = bi_gene_prop, fill = cT)) + 
# geom_bar(stat = "identity", position = position_dodge()) +
# scale_fill_viridis(discrete = TRUE) +
# theme_bw(base_size = 20) +
# xlab("") +
# ylab("% of Genes") +
# rremove("legend.title")

# ggarrange(p1, p2, p3, p4, ncol = 2, nrow = 2, common.legend = TRUE) 


b1 = bi_num %>% filter(cT != "EMRA") %>% ggplot(aes(x = cT, y = bi_peak_num, fill = cT)) + 
geom_boxplot() +
scale_fill_viridis(discrete = TRUE, begin = 0, end = 0.75, alpha = 0.8) +
theme_bw(base_size = 20) +
xlab("") +
ylab("# of Peak Regions") +
rremove("legend")

b2 = bi_num %>% filter(cT != "EMRA") %>% mutate(bi_peak_prop = bi_peak_num/total_peak_num * 100) %>% ggplot(aes(x = cT, y = bi_peak_prop, fill = cT)) + 
geom_boxplot() +
scale_fill_viridis(discrete = TRUE, begin = 0, end = 0.75, alpha = 0.8) +
theme_bw(base_size = 20) +
xlab("") +
ylab("% of Peak Regions") +
rremove("legend")

b3 = bi_num %>% filter(cT != "EMRA") %>% ggplot() + 
geom_boxplot(aes(x = cT, y = bi_gene_num, fill = cT)) +
scale_fill_viridis(discrete = TRUE, begin = 0, end = 0.75, alpha = 0.8) +
theme_bw(base_size = 20) +
xlab("") +
ylab("# of Genes") +
rremove("legend")


b4 = bi_num %>% filter(cT != "EMRA") %>% mutate(bi_gene_prop = bi_gene_num/total_gene_num * 100) %>% ggplot() + 
geom_boxplot(aes(x = cT, y = bi_gene_prop, fill = cT)) +
scale_fill_viridis(discrete = TRUE, begin = 0, end = 0.75, alpha = 0.8) +
theme_bw(base_size = 20) +
xlab("") +
ylab("% of Genes") +
rremove("legend") 

ggarrange(b1, b2, b3, b4, ncol = 4, nrow = 1)
# ggsave(paste0(fig_path, "/bivalent_peak_gene_num_prop_boxplot.pdf"), width = 28, height = 7)


# pdf(paste0(fig_path, "/bivalent_peak_gene_num_prop.pdf"), width = 10, height = 10)
# ggarrange(p1, p2, p3, p4, ncol = 2, nrow = 2, common.legend = TRUE) 
# dev.off()


In [ ]:
load(file = paste0(out_path, "/RData/HD1-3_5-7_normLimma_perExprCondition.RData"))
rna_results <- results

bi_change <- c()
bi_change_gene_lfc <- c()
ct_list <- c("N", "CM", "EM", "EMRA")
pt_list <- paste0("HD", 4:7)
for(i in 1:length(ct_list)){
    cT <- ct_list[i]
    cT1 <- cT
    for(j in 1:length(ct_list)){
        if(i != j){
            cT2 <- ct_list[j]
            for(pt in pt_list){
                bi_olp <- findOverlaps(peakAll[[paste0("H3K27me3_", cT, "_Input_", pt)]], peakAll[[paste0("H3K4me2_", cT, "_Input_", pt)]], minoverlap = 50)
                bi_olp2 <- findOverlaps(peakAll[[paste0("H3K27me3_", cT2, "_Input_", pt)]], peakAll[[paste0("H3K4me2_", cT2, "_Input_", pt)]], minoverlap = 50)

                bi_peak.gr <- reduce(append(peakAll[[paste0("H3K27me3_", cT, "_Input_", pt)]][bi_olp@from], peakAll[[paste0("H3K4me2_", cT, "_Input_", pt)]][bi_olp@to]))
                bi_peak.gr2 <- reduce(append(peakAll[[paste0("H3K27me3_", cT2, "_Input_", pt)]][bi_olp2@from], peakAll[[paste0("H3K4me2_", cT2, "_Input_", pt)]][bi_olp2@to]))

                bi_again <- findOverlaps(bi_peak.gr, bi_peak.gr2, minoverlap = 50)@from %>% unique
                bi_k27 <- findOverlaps(bi_peak.gr, peakAll[[paste0("H3K27me3_", cT2, "_Input_", pt)]], minoverlap = 50)@from %>% unique
                bi_k4 <- findOverlaps(bi_peak.gr, peakAll[[paste0("H3K4me2_", cT2, "_Input_", pt)]], minoverlap = 50)@from %>% unique
                bi_k27 <- setdiff(setdiff(bi_k27, bi_k4), bi_again)
                bi_k4 <- setdiff(setdiff(bi_k4, bi_k27), bi_again)
                bi_none <- setdiff(1:length(bi_peak.gr), unique(c(bi_again, bi_k27, bi_k4)) %>% sort)

                bi_gene <- get_nearest_gene(gene_gr, bi_peak.gr)
                bi_again_gene <- bi_gene %>% filter(peak_id %in% bi_again) %$% gene_name %>% unique
                bi_k27_gene <- bi_gene %>% filter(peak_id %in% bi_k27) %$% gene_name %>% unique
                bi_k4_gene <- bi_gene %>% filter(peak_id %in% bi_k4) %$% gene_name %>% unique
                bi_none_gene <- setdiff(unique(bi_gene$gene_name), c(bi_again_gene, bi_k27_gene, bi_k4_gene)) %>% unique
                # write.table(bi_again_gene, paste0(bi_path, "BivalentPeaksOf", cT1, "_BivalentPeakAgainIn", cT2, "_nearest_genes_", pt, ".csv"), quote = F, row.names = F, col.names = F, sep = ",")
                # write.table(bi_k27_gene, paste0(bi_path, "BivalentPeaksOf", cT1, "_H3K27me3In", cT2, "_nearest_genes_", pt, ".csv"), quote = F, row.names = F, col.names = F, sep = ",")
                # write.table(bi_k4_gene, paste0(bi_path, "BivalentPeaksOf", cT1, "_H3K4me2In", cT2, "_nearest_genes_", pt, ".csv"), quote = F, row.names = F, col.names = F, sep = ",")
                # write.table(bi_none_gene, paste0(bi_path, "BivalentPeaksOf", cT1, "_NoPeakIn", cT2, "_nearest_genes_", pt, ".csv"), quote = F, row.names = F, col.names = F, sep = ",")
                
                if(i < j){
                    bi_again_gene_lfc <- data.frame(gene_name = rna_results[[paste0("Input_", cT1, "_", cT2)]]$GeneName %>% gsub(".*_", "", .), GEX = rna_results[[paste0("Input_", cT1, "_", cT2)]]$logFC) %>% filter(gene_name %in% bi_again_gene)
                    bi_k27_gene_lfc <- data.frame(gene_name = rna_results[[paste0("Input_", cT1, "_", cT2)]]$GeneName %>% gsub(".*_", "", .), GEX = rna_results[[paste0("Input_", cT1, "_", cT2)]]$logFC) %>% filter(gene_name %in% bi_k27_gene)
                    bi_k4_gene_lfc <- data.frame(gene_name = rna_results[[paste0("Input_", cT1, "_", cT2)]]$GeneName %>% gsub(".*_", "", .), GEX = rna_results[[paste0("Input_", cT1, "_", cT2)]]$logFC) %>% filter(gene_name %in% bi_k4_gene)
                    bi_none_gene_lfc <- data.frame(gene_name = rna_results[[paste0("Input_", cT1, "_", cT2)]]$GeneName %>% gsub(".*_", "", .), GEX = rna_results[[paste0("Input_", cT1, "_", cT2)]]$logFC) %>% filter(gene_name %in% bi_none_gene)
                }else{
                    bi_again_gene_lfc <- data.frame(gene_name = rna_results[[paste0("Input_", cT2, "_", cT1)]]$GeneName %>% gsub(".*_", "", .), GEX = rna_results[[paste0("Input_", cT2, "_", cT1)]]$logFC) %>% filter(gene_name %in% bi_again_gene)
                    bi_k27_gene_lfc <- data.frame(gene_name = rna_results[[paste0("Input_", cT2, "_", cT1)]]$GeneName %>% gsub(".*_", "", .), GEX = rna_results[[paste0("Input_", cT2, "_", cT1)]]$logFC) %>% filter(gene_name %in% bi_k27_gene)
                    bi_k4_gene_lfc <- data.frame(gene_name = rna_results[[paste0("Input_", cT2, "_", cT1)]]$GeneName %>% gsub(".*_", "", .), GEX = rna_results[[paste0("Input_", cT2, "_", cT1)]]$logFC) %>% filter(gene_name %in% bi_k4_gene)
                    bi_none_gene_lfc <- data.frame(gene_name = rna_results[[paste0("Input_", cT2, "_", cT1)]]$GeneName %>% gsub(".*_", "", .), GEX = rna_results[[paste0("Input_", cT2, "_", cT1)]]$logFC) %>% filter(gene_name %in% bi_none_gene)

                }
                
                bi_change <- data.frame(
                    cT = cT, cT_compare = cT2, pt = pt, 
                    bi_original_num = length(bi_peak.gr), bi_again_num = length(bi_again), bi_k27_num = length(bi_k27), bi_k4_num = length(bi_k4), bi_none_num = length(bi_none),
                    bi_original_gene_num = length(unique(bi_gene$gene_name)), bi_again_gene_num = length(bi_again_gene), bi_k27_gene_num = length(bi_k27_gene), bi_k4_gene_num = length(bi_k4_gene), bi_none_gene_num = length(bi_none_gene)) %>% 
                    mutate(bi_again_prop = bi_again_num/bi_original_num * 100, bi_k27_prop = bi_k27_num/bi_original_num * 100, bi_k4_prop = bi_k4_num/bi_original_num * 100, bi_none_prop = bi_none_num/bi_original_num * 100) %>% 
                    mutate(bi_again_gene_prop = bi_again_gene_num/bi_original_gene_num * 100, bi_k27_gene_prop = bi_k27_gene_num/bi_original_gene_num * 100, bi_k4_gene_prop = bi_k4_gene_num/bi_original_gene_num * 100, bi_none_gene_prop = bi_none_gene_num/bi_original_gene_num * 100) %>% 
                    rbind(bi_change, .)
                
                bi_change_gene_lfc <- data.frame(
                    cT = cT, cT_compare = cT2, pt = pt, 
                    logFC = bi_again_gene_lfc$GEX, gene_name = bi_again_gene_lfc$gene_name, type = "bi_again"
                ) %>% rbind(bi_change_gene_lfc, .)
                bi_change_gene_lfc <- data.frame(
                    cT = cT, cT_compare = cT2, pt = pt, 
                    logFC = bi_k27_gene_lfc$GEX, gene_name = bi_k27_gene_lfc$gene_name, type = "bi_k27"
                ) %>% rbind(bi_change_gene_lfc, .)
                bi_change_gene_lfc <- data.frame(
                    cT = cT, cT_compare = cT2, pt = pt, 
                    logFC = bi_k4_gene_lfc$GEX, gene_name = bi_k4_gene_lfc$gene_name, type = "bi_k4"
                ) %>% rbind(bi_change_gene_lfc, .)
                if(nrow(bi_none_gene_lfc) > 0){
                    bi_change_gene_lfc <- data.frame(
                    cT = cT, cT_compare = cT2, pt = pt, 
                    logFC = bi_none_gene_lfc$GEX, gene_name = bi_none_gene_lfc$gene_name, type = "bi_none"
                ) %>% rbind(bi_change_gene_lfc, .)
                }
                
            }
        }
        
    }
}


bi_change %>% head
bi_change_gene_lfc %>% head

In [ ]:
library(tidyr)
bi_change$cT <- factor(bi_change$cT, levels = ct_list)
bi_change$cT_compare <- factor(bi_change$cT_compare, levels = ct_list)


p1 = bi_change %>% pivot_longer(., cols = c("bi_again_prop", "bi_k27_prop", "bi_k4_prop", "bi_none_prop")) %>% dplyr::select(name, value, cT, cT_compare, pt) %>%
filter(cT != "EMRA", cT_compare != "EMRA", name != "bi_none_prop") %>%
ggplot(aes(cT_compare, y = value, fill = factor(name, levels = c("bi_again_prop", "bi_k27_prop", "bi_k4_prop"), labels = c("Bivalent->Bivalent", "Bivalent->H3K27me3", "Bivalent->H3K4me2")), label = round(value, 1))) +
geom_boxplot(position = position_dodge(width = 0.9)) +
stat_summary(
    fun.y = median,
    geom = 'line',
    aes(group = name, colour = name),
    position = position_dodge(width = 0.9) #this has to be added
  ) +
facet_grid(~cT, scale = "free_x", space = "free") +
theme_bw(base_size = 20) +
scale_fill_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 0.8) +
scale_color_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1) +
xlab("") +
ylab("% of Peaks") +
rremove("legend.title") +
theme(legend.position = "top")

p12 = bi_change %>% pivot_longer(., cols = c("bi_again_prop", "bi_k27_prop", "bi_k4_prop", "bi_none_prop")) %>% dplyr::select(name, value, cT, cT_compare, pt) %>%
filter(cT != "EMRA", cT_compare != "EMRA", name != "bi_none_prop") %>%
ggplot(aes(cT_compare, y = value, fill = factor(name, levels = c("bi_again_prop", "bi_k27_prop", "bi_k4_prop"), labels = c("Bivalent->Bivalent", "Bivalent->H3K27me3", "Bivalent->H3K4me2")), label = round(value, 1))) +
geom_boxplot(position = position_dodge(width = 0.9)) +
facet_grid(~cT, scale = "free_x", space = "free") +
theme_bw(base_size = 20) +
scale_fill_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 0.8) +
scale_color_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1) +
xlab("") +
ylab("% of Peaks") +
rremove("legend.title") +
theme(legend.position = "top")

p2 = bi_change %>% pivot_longer(., cols = c("bi_again_gene_prop", "bi_k27_gene_prop", "bi_k4_gene_prop", "bi_none_gene_prop")) %>% dplyr::select(name, value, cT, cT_compare, pt) %>%
filter(cT != "EMRA", cT_compare != "EMRA", name != "bi_none_gene_prop") %>%
ggplot(aes(cT_compare, y = value, fill = factor(name, levels = c("bi_again_gene_prop", "bi_k27_gene_prop", "bi_k4_gene_prop"), labels = c("Bivalent->Bivalent", "Bivalent->H3K27me3", "Bivalent->H3K4me2")), label = round(value, 1))) +
geom_boxplot(position = position_dodge(width = 0.9)) +
stat_summary(
    fun.y = median,
    geom = 'line',
    aes(group = name, colour = name),
    position = position_dodge(width = 0.9) #this has to be added
  ) +
facet_grid(~cT, scale = "free_x", space = "free") +
theme_bw(base_size = 20) +
scale_fill_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 0.8) +
scale_color_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1) +
xlab("") +
ylab("% of Genes") +
rremove("legend.title") +
theme(legend.position = "top")

p22 = bi_change %>% pivot_longer(., cols = c("bi_again_gene_prop", "bi_k27_gene_prop", "bi_k4_gene_prop", "bi_none_gene_prop")) %>% dplyr::select(name, value, cT, cT_compare, pt) %>%
filter(cT != "EMRA", cT_compare != "EMRA", name != "bi_none_gene_prop") %>%
ggplot(aes(cT_compare, y = value, fill = factor(name, levels = c("bi_again_gene_prop", "bi_k27_gene_prop", "bi_k4_gene_prop"), labels = c("Bivalent->Bivalent", "Bivalent->H3K27me3", "Bivalent->H3K4me2")), label = round(value, 1))) +
geom_boxplot(position = position_dodge(width = 0.9)) +
facet_grid(~cT, scale = "free_x", space = "free") +
theme_bw(base_size = 20) +
scale_fill_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 0.8) +
scale_color_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1) +
xlab("") +
ylab("% of Genes") +
rremove("legend.title") +
theme(legend.position = "top")

# pdf(paste0(fig_path, "/bivalent_peak_gene_change_between_celltypes.pdf"), width = 10, height = 8)
# p1
# p12
# p2
# p22
# dev.off()

ggarrange(p12, p22, nrow = 2, ncol = 1, common.legend = TRUE)


In [ ]:
bi_change %>% 
    pivot_longer(., cols = c("bi_again_prop", "bi_k27_prop", "bi_k4_prop", "bi_none_prop")) %>%
    colnames

In [ ]:
bi_change[c("bi_again_prop", "bi_k27_prop", "bi_k4_prop", "bi_none_prop")]

In [ ]:
bi_change_gene_lfc %>% head
bi_change_gene_lfc$cT <- factor(bi_change_gene_lfc$cT, levels = ct_list)
bi_change_gene_lfc$cT_compare <- factor(bi_change_gene_lfc$cT_compare, levels = ct_list)

bi_change_gene_lfc %>% filter(cT != "EMRA", cT_compare != "EMRA", type != "bi_none") %>%
ggplot(aes(cT_compare, y = logFC, fill = factor(type, levels = c("bi_again", "bi_k27", "bi_k4"), labels = c("Bivalent->Bivalent", "Bivalent->H3K27me3", "Bivalent->H3K4me2")))) +
geom_boxplot() +
facet_grid(~cT, scale = "free_x", space = "free") +
theme_bw(base_size = 20) +
scale_fill_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 0.8) +
xlab("") +
ylab("logFC of GEX") +
rremove("legend.title") +
theme(legend.position = "top")

# ggsave(paste0(fig_path, "/bivalent_peak_gene_change_between_celltypes_gex_logfc.pdf"), width = 10, height = 8)


In [ ]:
library(dplyr)
library(ggplot2)
library(rstatix)
# tmp = bi_change_gene_lfc
# bi_change_gene_lfc = tmp
# bi_change_gene_lfc$type = factor(bi_change_gene_lfc$type, levels = c("bi_again", "bi_k27", "bi_k4", "bi_none"), 
#              labels = c("Bivalent->Bivalent", "Bivalent->H3K27me3", "Bivalent->H3K4me2", "Bivalent->None"))

bi_change_gene_lfc %>% 
  filter(cT != "EMRA", cT_compare != "EMRA", type != "Bivalent->None") %>%
  ggplot(aes(cT_compare, y = logFC, fill = type)) +
  geom_boxplot() +
  facet_grid(~cT, scale = "free_x", space = "free") +
  stat_compare_means(method = "anova", label = "p.format", 
                     symnum.args = list(cutpoints = c(0, 0.001, 0.01, 0.05, 1), 
                                        symbols = c("***", "**", "*", "ns"))) + 
  theme_bw(base_size = 20) +
  scale_fill_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 0.8) +
  xlab("") +
  ylab("logFC of GEX") +
  rremove("legend.title") +
  theme(legend.position = "top")

ggsave(paste0(fig_path, "/bivalent_peak_gene_change_between_celltypes_gex_logfc_withANOVAtestPValues.pdf"), width = 10, height = 8)

In [ ]:
lfc_thres = 1
bi_change_gene_lfc$logFC_group = "nonDE"
bi_change_gene_lfc$logFC_group[bi_change_gene_lfc$logFC < -lfc_thres] = "down_regulated"
bi_change_gene_lfc$logFC_group[bi_change_gene_lfc$logFC > lfc_thres] = "up_regulated"
tmp1 = bi_change_gene_lfc %>% group_by(cT, cT_compare, pt, type, logFC_group) %>% summarize(gene_num = n())
tmp2 = bi_change_gene_lfc %>% group_by(cT, cT_compare, pt, type) %>% summarize(gene_total_num = n())
bi_change_gene_lfc_group = left_join(tmp1, tmp2, by = c("cT", "cT_compare", "pt", "type")) %>% mutate(gene_prop = gene_num/gene_total_num * 100)

In [ ]:
options(repr.plot.width=25, repr.plot.height=8)
fig_path = "CART_CUTRUN_Project/results/paper_figure/fig5"

p_gene_prop = bi_change_gene_lfc_group %>% filter(cT != "EMRA", cT_compare != "EMRA", type != "bi_none") %>% ggplot(aes(x = cT_compare, y = gene_prop, fill = type)) +
geom_boxplot(aes(alpha = logFC_group)) +
scale_fill_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 0.5) +
# scale_color_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 1) +
theme_bw(base_size = 30) +
facet_grid(~cT, scales = "free") +
ylab("% of Genes") +
scale_alpha_discrete(range = c(0.3, 1)) +
xlab("") +
rremove("legend")

p_gene_prop
ggsave(paste0(fig_path, "/bivalent_gene_expression_change_proportion_logFC_thres1.pdf"), width = 9, height = 8)


In [ ]:
p22
ggsave(paste0(fig_path, "/bivalent_peak_gene_change_between_celltypes_geneprop.pdf"), width = 9, height = 8)


In [ ]:
lfc_thres = 2
bi_change_gene_lfc$logFC_group = "nonDE"
bi_change_gene_lfc$logFC_group[bi_change_gene_lfc$logFC < -lfc_thres] = "DE"
bi_change_gene_lfc$logFC_group[bi_change_gene_lfc$logFC > lfc_thres] = "DE"
tmp1 = bi_change_gene_lfc %>% group_by(cT, cT_compare, pt, type, logFC_group) %>% summarize(gene_num = n())
tmp2 = bi_change_gene_lfc %>% group_by(cT, cT_compare, pt) %>% summarize(gene_total_num = n())
bi_change_gene_lfc_group = left_join(tmp1, tmp2, by = c("cT", "cT_compare", "pt")) %>% mutate(gene_prop = gene_num/gene_total_num * 100)
bi_change_gene_lfc_group$type = factor(bi_change_gene_lfc_group$type, levels = c("bi_again", "bi_k27", "bi_k4"), labels = c("Bivalent->Bivalent", "Bivalent->H3K27me3", "Bivalent->H3K4me2"))

bi_change_gene_lfc_group %>% head

In [ ]:

p_gene_prop = bi_change_gene_lfc_group %>% filter(cT != "EMRA", cT_compare != "EMRA", type != "bi_none", logFC_group == "DE") %>% 
ggplot(aes(x = cT_compare, y = gene_prop, fill = type)) +
geom_boxplot() +
scale_fill_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 0.8) +
# scale_color_viridis(discrete = TRUE, option = "magma", begin = 0.2, end = 1, alpha = 1) +
theme_bw(base_size = 20) +
facet_grid(~cT, scales = "free") +
ylab("% of Genes") +
scale_alpha_discrete(range = c(0.3, 1)) +
xlab("") +
theme(legend.position = "top") +
ggpubr::rremove("legend.title") +
ylim(0, 60)

p_gene_prop
ggsave(paste0(fig_path, "/bivalent_gene_DE_proportion_logFC_thres2.pdf"), width = 9, height = 9)


In [ ]:
## PCA for RNA and CUTRUN
load(file = paste0(outPath, "/RData/countMat_designInfo_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData")) ## 'cell_comp_list''cell_list''countMat''designInfo''expr_list''fig_path''hd_list''hist_list''in_path''inPath''out_path''outPath'
load(file = paste0(outPath, "/RData/masterPeak_peakAnno_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = "CART_CUTRUN_Project/results/RNAseq/analysis/RSEM/RData/HD1-3_5-7_normLimma_perExprCondition.RData") ## data, selectR, dataS, voomDDS, results, 

geneID = fread(paste0("CART_CUTRUN_Project/results/RNAseq/process/RSEM/", "RNA_CD8_N_Input_HD1.genes.results"))$gene_id
data = data.frame(data)
select_exp = "Product"

rna_m = data[, (colnames(data) %>% stringr::str_detect(select_exp)) & !(colnames(data) %>% stringr::str_detect("EMRA"))  & !(colnames(data) %>% stringr::str_detect("N"))]
k27_m = countMat[["H3K27me3"]][, (colnames(countMat[["H3K27me3"]]) %>% stringr::str_detect(select_exp)) & !(colnames(countMat[["H3K27me3"]]) %>% stringr::str_detect("EMRA")) & !(colnames(countMat[["H3K27me3"]]) %>% stringr::str_detect("N"))]
k4_m = countMat[["H3K4me2"]][, (colnames(countMat[["H3K4me2"]]) %>% stringr::str_detect(select_exp)) & !(colnames(countMat[["H3K4me2"]]) %>% stringr::str_detect("EMRA")) & !(colnames(countMat[["H3K4me2"]]) %>% stringr::str_detect("N"))]

rownames(rna_m) = geneID
rownames(k27_m) = mPeak[["H3K27me3"]] %>% data.frame %>% mutate(region = paste0(seqnames, ":", start, "-", end)) %$% region
rownames(k4_m) = mPeak[["H3K4me2"]] %>% data.frame %>% mutate(region = paste0(seqnames, ":", start, "-", end)) %$% region

dim(rna_m)
dim(k27_m)
dim(k4_m)
head(rna_m)
head(k27_m)
head(k4_m)

rna_voom = voomDDS$E[, colnames(voomDDS$E) %>% stringr::str_detect(select_exp)]
load(file = paste0(outPath, "/RData/results_histList_hd1-7_adjustPeak_noChrXYM.RData"))
k27_voom = voomDDS[["H3K27me3"]]$E[, colnames(voomDDS[["H3K27me3"]]$E) %>% stringr::str_detect(select_exp)]
k4_voom = voomDDS[["H3K4me2"]]$E[, colnames(voomDDS[["H3K4me2"]]$E) %>% stringr::str_detect(select_exp)]
rownames(rna_voom) = 1:nrow(rna_voom)
rownames(k27_voom) = paste0("K27me3_", 1:nrow(k27_voom))
rownames(k4_voom) = paste0("K4me2_", 1:nrow(k4_voom))

dim(rna_voom)
dim(k27_voom)
dim(k4_voom)

In [ ]:
rna_obj_tmp = CreateSeuratObject(counts = rna_m, project = "cart", min.cells = 0, min.features = 10) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) 

rna_obj = rna_obj_tmp %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 1000) %>% 
ScaleData()
rna_obj = RunPCA(rna_obj, features = VariableFeatures(object = rna_obj), npcs = 10)

k27_obj_tmp = CreateSeuratObject(counts = k27_m, project = "cart", min.cells = 0, min.features = 10) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000)

k27_obj = k27_obj_tmp %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 10000) %>%  #
ScaleData()

k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 12)

k4_obj_tmp = CreateSeuratObject(counts = k4_m, project = "cart", min.cells = 0, min.features = 10) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) 
k4_obj = k4_obj_tmp %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 1000) %>%  #round(nrow(k4_m)/10*8)
ScaleData()
k4_obj = RunPCA(k4_obj, features = VariableFeatures(object = k4_obj), npcs = 12)

# cr_obj = CreateSeuratObject(counts = rbind(k27_m, k4_m), project = "cart", min.cells = 0, min.features = 10) %>%  
# NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
# FindVariableFeatures(., selection.method = "vst", nfeatures = 500) %>%  #round((nrow(k27_m) + nrow(k4_m))/10*8)
# ScaleData()
# cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 12)


cr_obj = CreateSeuratObject(counts = rbind(k27_obj_tmp@assays$RNA@data, k4_obj_tmp@assays$RNA@data), project = "cart", min.cells = 0, min.features = 10) %>%  
# NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 1000) %>%  #round((nrow(k27_m) + nrow(k4_m))/10*8)
ScaleData()
cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 12)


In [ ]:
options(repr.plot.width=20, repr.plot.height=7)
# DimPlot(rna_obj, reduction = "pca")
ct_list = c("CM", "EM")
p1 = Embeddings(rna_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(hd = factor(rep(paste0("HD", c(1, 2, 3, 5, 6, 7)), 2))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = hd)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Paired") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("RNA-seq") +
rremove("legend.title")+ theme(legend.position = "top")

p2 = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(hd = factor(rep(paste0("HD", 1:7), 2))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = hd)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Paired") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
rremove("legend.title")+ theme(legend.position = "top")

p3 = Embeddings(k4_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(hd = factor(rep(paste0("HD", 1:7), 2))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = hd)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Paired") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
rremove("legend.title")+ theme(legend.position = "top")

p4 = Embeddings(cr_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(hd = factor(rep(paste0("HD", 1:7), 2))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = hd)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_brewer(palette = "Paired") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3+H3K4me2") +
rremove("legend.title")+ theme(legend.position = "top")

ggarrange(p1, p2, p3, p4, nrow = 1, ncol = 4, common.legend = FALSE) 
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig6/PCA_seurat_procedure_CAR-T_Product_CM_EM_colorbyHD.pdf", width = 20, height = 7)

In [ ]:
options(repr.plot.width=15, repr.plot.height=5)
# DimPlot(rna_obj, reduction = "pca")
ct_list = c("CM", "EM")
p1 = Embeddings(rna_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(celltype = factor(rep(ct_list, each = 6), levels = ct_list)) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = celltype)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.2, end = 0.75) +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("RNA-seq") +
rremove("legend.title")

p2 = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(celltype = factor(rep(ct_list, each = 7), levels = ct_list)) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = celltype)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.2, end = 0.75) +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
rremove("legend.title")

p3 = Embeddings(k4_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(celltype = factor(rep(ct_list, each = 7), levels = ct_list)) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = celltype)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.2, end = 0.75) +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
rremove("legend.title")

p4 = Embeddings(cr_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(celltype = factor(rep(ct_list, each = 7), levels = ct_list)) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = celltype)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.2, end = 0.75) +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3+H3K4me2") +
rremove("legend.title")

ggarrange(p1, p2, p3, p4, nrow = 1, ncol = 4, common.legend = TRUE)
# ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig6/PCA_seurat_procedure_CAR-T_Product_CM_EM.pdf", width = 20, height = 6)

In [ ]:
## patient related

## Patient data infusion product of CM-derived cells 
patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
cm_ind = which(patientRaw$`CD8+ T cell isolation` == "CD8TCM")
patient_id_list = patientRaw$Xnumber[cm_ind] %>% tolower
patient_id_list

## Patient data infusion product of CM-derived cells 
inPath = "CART_CUTRUN_Project/results/RNAseq/process/NHL_RSEM_gencode_v21/"

pt_expCount = c()
pt_name_tmp_list = c()
for(sample in paste0("Sample_", patient_id_list, "_IP_CD8-CAR")){
  if(file.exists(paste0(inPath, sample, "/RSEM.genes.results"))){
    print(sample)
    tmp = fread(paste0(inPath, sample, "/RSEM.genes.results")) %>% dplyr::select(expected_count)
    pt_expCount = cbind(pt_expCount, round(tmp))
    pt_name_tmp_list = c(pt_name_tmp_list, sample)
  }
}
pt_expCount = data.frame(pt_expCount)
colnames(pt_expCount) = paste0(pt_name_tmp_list, "_PT") #paste0(patient_id_list, "_PT")

geneID = fread(paste0(inPath, sample, "/RSEM.genes.results"))$gene_id
pt_expCount$gene_name = geneID

head(pt_expCount)
dim(pt_expCount)

## healthy donor data
## Read in RNA-seq data
cellList <- "CM" #c("N", "CM", "EM", "EMRA")
hdList <- paste0("HD", c(1:3, 5:7))
exprList = "Product" #c("Input", "Product", "Stim1", "Stim2", "Stim3")

inPath <- "CART_CUTRUN_Project/results/RNAseq/process/HD_RSEM_gencode_v21/" #"CART_CUTRUN_Project/results/RNAseq/process/RSEM/"

hd_expCount = c()
for(cell in cellList){
  for(expr in exprList){
    for(hd in hdList){
      # print(paste(expr, cell, hd, sep = "_"))
      sample = paste0("RNA_CD8_", cell, "_", expr, "_", hd)
      if(file.exists(paste0(inPath, sample, "/RSEM.genes.results"))){
        print(sample)
        dataTmp = fread(paste0(inPath, sample, "/RSEM.genes.results")) %>% dplyr::select(expected_count)
        # dataTmp <- fread(paste0(inPath, "RNA_CD8_", cell, "_", expr, "_", hd, ".genes.results")) %>% dplyr::select(expected_count)
        hd_expCount <- cbind(hd_expCount, round(dataTmp))
      }

    }
  }

}
hd_expCount = data.frame(hd_expCount)
colnames(hd_expCount) <- paste(rep(rep(exprList, each = length(hdList)), length(cellList)), rep(cellList, each = length(hdList)*length(exprList)), rep(hdList, length(exprList)*length(cellList)), sep = "_")

geneID = fread(paste0(inPath, sample, "/RSEM.genes.results"))$gene_id
hd_expCount$gene_name <- geneID
head(hd_expCount)
dim(hd_expCount)

dim(pt_expCount)
dim(hd_expCount)
countMat = inner_join(pt_expCount, hd_expCount, by = "gene_name")
rownames(countMat) = countMat$gene_name
countMat = countMat %>% dplyr::select(-gene_name)
head(countMat)
dim(countMat)

countMat_rna = countMat

In [ ]:
options(repr.plot.width=7, repr.plot.height=6)
selectR = which(rowSums(countMat_rna) > 10) ## remove low count genes
length(selectR)

rna_obj = CreateSeuratObject(counts = countMat_rna[selectR, ], project = "cart", min.cells = 0, min.features = 10) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = 50000) %>%  # length(selectR)
ScaleData()

rna_obj = RunPCA(rna_obj, features = VariableFeatures(object = rna_obj), npcs = 10)
rna_plot = Embeddings(rna_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(group = c(rep("PT", ncol(pt_expCount) - 1), rep("HD", ncol(hd_expCount) - 1))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = group)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.2, end = 0.8, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("RNA-seq") +
rremove("legend.title")

# pdf("CART_CUTRUN_Project/results/paper_figure/fig1/CMpatient_CMhealthyDonor_PCA_on_RNA-seq_allfeatures.pdf", width = 7, height = 6)
# rna_plot
# dev.off()
rna_plot

In [ ]:
patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
cm_ind = which(patientRaw$`CD8+ T cell isolation` == "CD8TCM")
patientRaw$Xnumber[cm_ind]

## 1. TSS neighbor regions
## use TSS region for differential detection
## Get the TSS location and +/-1kb region around TSS
gtf <- rtracklayer::import('/fh/fast/gottardo_r/yezheng_working/SupplementaryData/hg38/transcriptomeHumanReference/gencode.v33.annotation.gtf') #rtracklayer::import('/fh/fast/gottardo_r/yezheng_working/SupplementaryData/hg38/GENCODE/gencode.v21.annotation.gtf')
gtf_df=as.data.frame(gtf)


## get the TSS location and extend by 1kb up and downstream
tss_df = rbind(
    gtf_df %>% data.frame %>% dplyr::filter(type == "gene", strand == "+") %>% dplyr::select(seqnames, TSS = start, strand, source, gene_id, gene_name, gene_name, gene_type) ,
    gtf_df %>% data.frame %>% dplyr::filter(type == "gene", strand == "-") %>% dplyr::select(seqnames, TSS = end, strand, source, gene_id, gene_name, gene_name, gene_type) 
)
tss_filter_df = tss_df %>% dplyr::filter(!(seqnames %in% c("chrM", "chrX", "chrY")))

tss_gr = GRanges(seqnames = tss_filter_df$seqnames, ranges = IRanges(start = tss_filter_df$TSS - 1000, end = tss_filter_df$TSS + 1000), strand = tss_filter_df$strand, gene_id = tss_filter_df$gene_id, gene_name = tss_filter_df$gene_name, gene_type = tss_filter_df$gene_type)
tss_gr 

## patient data

outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"
histList = c("H3K27me3", "H3K4me2")

countMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_countMat.rds"))
normMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_normMat.rds"))
designInfo = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_designInfo.rds"))

pt_countMat = countMat
pt_countMat$H3K27me3 = countMat$H3K27me3[, cm_ind]
pt_countMat$H3K4me2 = countMat$H3K4me2[, cm_ind]
colnames(pt_countMat$H3K27me3) = paste0(colnames(pt_countMat$H3K27me3), "_PT")
colnames(pt_countMat$H3K4me2) = paste0(colnames(pt_countMat$H3K4me2), "_PT")

pt_designInfo = designInfo %>% dplyr::filter(exps %in% patientRaw$Xnumber[cm_ind])
head(pt_countMat$H3K27me3)
pt_designInfo %>% arrange(hist, exps)

## healthy donor data
countMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_histList_hd1-7_noChrXYM_countMat.rds"))
normMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_histList_hd1-7_noChrXYM_normMat.rds"))
designInfo = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_histList_hd1-7_noChrXYM_designInfo.rds"))

selectC = colnames(countMat$H3K27me3) %>% stringr::str_detect("CM_Product")
head(countMat$H3K4me2[, selectC])

hd_countMat = countMat
hd_countMat$H3K27me3 = countMat$H3K27me3[, selectC]
hd_countMat$H3K4me2 = countMat$H3K4me2[, selectC]

hd_designInfo = designInfo[which(stringr::str_detect(designInfo$exps, "CM_Product")), ]
hd_designInfo

for(hist in histList){
    tmp = c()

    for(j in 1:ncol(pt_countMat[[hist]])){
        pt_designInfo_tmp = pt_designInfo %>% dplyr::filter(hist == hist)
        tmp = cbind(tmp, round(pt_countMat[[hist]][, j]/pt_designInfo_tmp$depth[j] * 13240732)) ## remove sequencing depth effect
    }
    for(j in 1:ncol(hd_countMat[[hist]])){
        hd_designInfo_tmp = hd_designInfo %>% dplyr::filter(hist == hist)
        tmp = cbind(tmp, round(hd_countMat[[hist]][, j]/hd_designInfo_tmp$depth[j] * 13240732)) ## remove sequencing depth effect
    }
    countMat[[hist]] = tmp
    colnames(countMat[[hist]]) = c(colnames(pt_countMat[[hist]]), colnames(hd_countMat[[hist]]))
}


In [ ]:
options(repr.plot.width=22, repr.plot.height=10)
hist = "H3K27me3"
selectR = which(rowSums(countMat[[hist]]) > 1000) ## remove low count genes

tmp = countMat[[hist]][selectR, ]

k27_obj = CreateSeuratObject(counts = countMat[[hist]][selectR, ], project = "cart", min.cells = 0, min.features = 10) %>%  
# NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = round(length(selectR)/2)) %>% 
ScaleData()
k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 15)
k27_plot = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(group = c(rep("PT", 15), rep("HD", 7))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = group)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.2, end = 0.8, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
rremove("legend.title")

hist = "H3K4me2"
selectR = which(rowSums(countMat[[hist]]) > 1000) ## remove low count genes
length(selectR)
k4_obj = CreateSeuratObject(counts = countMat[[hist]][selectR, ], project = "cart", min.cells = 0, min.features = 10) %>%  
# NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = round(length(selectR)/2)) %>% 
ScaleData()
k4_obj = RunPCA(k4_obj, features = VariableFeatures(object = k4_obj), npcs = 15)

k4_plot = Embeddings(k4_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(group = c(rep("PT", 15), rep("HD", 7))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = group)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.2, end = 0.8, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
rremove("legend.title")

cr_obj = CreateSeuratObject(counts = rbind(tmp, countMat[[hist]][selectR, ]), project = "cart", min.cells = 0, min.features = 10) %>%  
# NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = round((length(selectR) + nrow(tmp))/1)) %>% 
ScaleData()
cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 15)
cr_plot = Embeddings(cr_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(group = c(rep("PT", 15), rep("HD", 7))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = group)) +
geom_point(size = 5) +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.2, end = 0.8, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3 + H3K4me2") +
rremove("legend.title")


pdf("CART_CUTRUN_Project/results/paper_figure/fig6/CMpatient_CMhealthyDonor_PCA_on_all_TSSneighbors.pdf", width = 15, height = 6)
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)
dev.off()
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)


In [ ]:
metadata_path = "CART_CUTRUN_Project/meta/"
clinical_data = fread(paste0(metadata_path, "NHL_DL2_FluCy_Propscore_linear_model_CandR_only_pfs.csv"), sep = ",", header = TRUE) %>% data.frame
head(clinical_data)

clinical_data %>% colnames
clinical_select = c("xnumber", "cd8_max", "cd4_max", "log_cd8_max", "propscore_cd8_max", "propscore", "median_cd8_max", "preld_ldh", "pfscode",	"pfsdaysafterfirstcartcell", "pfs_late_code", "log_pfs_days")
clinical_data_select = clinical_data[, clinical_select]
clinical_data_select

In [ ]:
marker_gene_table = fread(paste0(metadata_path, "Table_S3.csv"), sep = ",", header = TRUE)
marker_gene = marker_gene_table$GeneNameClean
k27_TF_gene = marker_gene_table %>% filter(Analysis == "K27") %$% GeneNameClean
k4_TF_gene = marker_gene_table %>% filter(Analysis == "K4") %$% GeneNameClean
rna_TF_gene = marker_gene_table %>% filter(Analysis == "RNA") %$% GeneNameClean

marker_gene_table

In [ ]:
patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
cm_ind = which(patientRaw$`CD8+ T cell isolation` == "CD8TCM")
patientRaw$Xnumber[cm_ind]

## patient data

outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"
histList = c("H3K27me3", "H3K4me2")

countMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_countMat.rds"))
normMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_normMat.rds")) ## RC
designInfo = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_designInfo.rds"))

# pt_countMat = countMat
# pt_countMat$H3K27me3 = countMat$H3K27me3[, cm_ind]
# pt_countMat$H3K4me2 = countMat$H3K4me2[, cm_ind]
# colnames(pt_countMat$H3K27me3) = paste0(colnames(pt_countMat$H3K27me3), "_PT")
# colnames(pt_countMat$H3K4me2) = paste0(colnames(pt_countMat$H3K4me2), "_PT")

pt_normMat = normMat
pt_normMat$H3K27me3 = log2(normMat$H3K27me3[, patientRaw$Xnumber[cm_ind]]+1)
pt_normMat$H3K4me2 = log2(normMat$H3K4me2[, patientRaw$Xnumber[cm_ind]]+1)
# colnames(pt_normMat$H3K27me3) = paste0(colnames(pt_normMat$H3K27me3), "_PT")
# colnames(pt_normMat$H3K4me2) = paste0(colnames(pt_normMat$H3K4me2), "_PT")


pt_designInfo = designInfo %>% dplyr::filter(exps %in% patientRaw$Xnumber[cm_ind])
head(pt_normMat$H3K27me3)
pt_designInfo %>% arrange(hist, exps)



In [ ]:
options(repr.plot.width=25, repr.plot.height=20)

pt_select_tmp = pt_normMat$H3K27me3[marker_gene, ] %>% data.frame
pt_select = pt_select_tmp %>% mutate(gene_num = rownames(pt_select_tmp)) %>% pivot_longer(cols = colnames(pt_select_tmp), names_to = "xnumber")
head(pt_select, 20)
pt_select %>% dim

pt_clinical = left_join(left_join(pt_select, clinical_data_select, by = "xnumber"), marker_gene_table[, c("Analysis", "GeneNameClean")], by = c("gene_num" = "GeneNameClean"))
pt_clinical %>% head

pt_clinical %>% ggplot(aes(x = as.factor(propscore_cd8_max), y = value)) +
geom_boxplot(aes(fill = Analysis)) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") + 
stat_compare_means(vjust = 1) +
scale_fill_brewer(palette = "Dark2") +
xlab("Propensity Score CD8 Max") +
ylab("H3K27me3: log2(Scaled Count + 1)")

ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K27me3_signal_patient_propscore_cd8_max.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% k27_TF_gene) %>% ggplot(aes(x = as.factor(propscore_cd8_max), y = value)) +
geom_boxplot(aes(fill = as.factor(propscore_cd8_max))) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") + 
stat_compare_means(vjust = 1, size = 6.5) +
scale_fill_brewer(palette = "Paired") +
xlab("Propensity Score CD8 Max") +
ylab("H3K27me3: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K27me3_signal_patient_propscore_cd8_max_k27DEgenes.pdf", width = 18, height = 18)

pt_clinical %>% ggplot(aes(x = log_cd8_max, y = value)) +
geom_point(aes(color = Analysis)) +
geom_smooth(aes(color = Analysis), method = "lm") +
theme_bw(base_size = 25) +
scale_color_brewer(palette = "Dark2") +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") +
xlab("Log CD8 Max") +
ylab("H3K27me3: log2(Scaled Count + 1)")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K27me3_signal_patient_log_cd8_max.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% k27_TF_gene) %>% ggplot(aes(x = log_cd8_max, y = value)) +
geom_point(aes(color = log_cd8_max)) +
geom_smooth(color = "grey", method = "lm", se = FALSE) +
theme_bw(base_size = 25) +
scale_color_viridis() +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") +
xlab("Log CD8 Max") +
ylab("H3K27me3: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K27me3_signal_patient_log_cd8_max_k27DEgenes.pdf", width = 20, height = 14)

## "pfscode",	"pfsdaysafterfirstcartcell", "pfs_late_code", "log_pfs_days"

options(repr.plot.width=25, repr.plot.height=20)

pt_clinical %>% ggplot(aes(x = as.factor(pfs_late_code), y = value)) +
geom_boxplot(aes(fill = Analysis)) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") + 
stat_compare_means(vjust = 1) +
scale_fill_brewer(palette = "Dark2") +
xlab("PFS Late Code") +
ylab("H3K27me3: log2(Scaled Count + 1)")

ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K27me3_signal_patient_PFS_late_code.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% k27_TF_gene) %>% ggplot(aes(x = as.factor(pfs_late_code), y = value)) +
geom_boxplot(aes(fill = as.factor(pfs_late_code))) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") + 
stat_compare_means(vjust = 1, size = 6.5) +
scale_fill_brewer(palette = "Paired") +
xlab("PFS Late Code") +
ylab("H3K27me3: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K27me3_signal_patient_PFS_late_code_k27DEgenes.pdf", width = 18, height = 18)

pt_clinical %>% ggplot(aes(x = log_pfs_days, y = value)) +
geom_point(aes(color = Analysis)) +
geom_smooth(aes(color = Analysis), method = "lm") +
theme_bw(base_size = 25) +
scale_color_brewer(palette = "Dark2") +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") +
xlab("Log PFS Days") +
ylab("H3K27me3: log2(Scaled Count + 1)")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K27me3_signal_patient_log_pfs_days.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% k27_TF_gene) %>% ggplot(aes(x = log_pfs_days, y = value)) +
geom_point(aes(color = log_pfs_days)) +
geom_smooth(color = "grey", method = "lm", se = FALSE) +
theme_bw(base_size = 25) +
scale_color_viridis() +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") +
xlab("Log PFS Days") +
ylab("H3K27me3: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K27me3_signal_patient_log_pfs_days_k27DEgenes.pdf", width = 20, height = 14)







In [ ]:
options(repr.plot.width=25, repr.plot.height=20)

pt_select_tmp = pt_normMat$H3K4me2[marker_gene, ] %>% data.frame
pt_select = pt_select_tmp %>% mutate(gene_num = rownames(pt_select_tmp)) %>% pivot_longer(cols = colnames(pt_select_tmp), names_to = "xnumber")
head(pt_select, 20)
pt_select %>% dim

pt_clinical = left_join(left_join(pt_select, clinical_data_select, by = "xnumber"), marker_gene_table[, c("Analysis", "GeneNameClean")], by = c("gene_num" = "GeneNameClean"))
pt_clinical %>% head

pt_clinical %>% ggplot(aes(x = as.factor(propscore_cd8_max), y = value)) +
geom_boxplot(aes(fill = Analysis)) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") + 
stat_compare_means(vjust = 1) +
scale_fill_brewer(palette = "Dark2") +
xlab("Propensity Score CD8 Max") +
ylab("H3K4me2: log2(Scaled Count + 1)")

ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K4me2_signal_patient_propscore_cd8_max.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% k4_TF_gene) %>% ggplot(aes(x = as.factor(propscore_cd8_max), y = value)) +
geom_boxplot(aes(fill = as.factor(propscore_cd8_max))) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") + 
stat_compare_means(vjust = 1, size = 6.5) +
scale_fill_brewer(palette = "Paired") +
xlab("Propensity Score CD8 Max") +
ylab("H3K4me2: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K4me2_signal_patient_propscore_cd8_max_k4DEgenes.pdf", width = 18, height = 18)

pt_clinical %>% ggplot(aes(x = log_cd8_max, y = value)) +
geom_point(aes(color = Analysis)) +
geom_smooth(aes(color = Analysis), method = "lm") +
theme_bw(base_size = 25) +
scale_color_brewer(palette = "Dark2") +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") +
xlab("Log CD8 Max") +
ylab("H3K4me2: log2(Scaled Count + 1)")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K4me2_signal_patient_log_cd8_max.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% k4_TF_gene) %>% ggplot(aes(x = log_cd8_max, y = value)) +
geom_point(aes(color = log_cd8_max)) +
geom_smooth(color = "grey", method = "lm", se = FALSE) +
theme_bw(base_size = 25) +
scale_color_viridis() +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") +
xlab("Log CD8 Max") +
ylab("H3K4me2: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K4me2_signal_patient_log_cd8_max_k4DEgenes.pdf", width = 20, height = 10)

## "pfscode",	"pfsdaysafterfirstcartcell", "pfs_late_code", "log_pfs_days"

options(repr.plot.width=25, repr.plot.height=20)

pt_clinical %>% ggplot(aes(x = as.factor(pfs_late_code), y = value)) +
geom_boxplot(aes(fill = Analysis)) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") + 
stat_compare_means(vjust = 1) +
scale_fill_brewer(palette = "Dark2") +
xlab("PFS Late Code") +
ylab("H3K4me2: log2(Scaled Count + 1)")

ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K4me2_signal_patient_PFS_late_code.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% k4_TF_gene) %>% ggplot(aes(x = as.factor(pfs_late_code), y = value)) +
geom_boxplot(aes(fill = as.factor(pfs_late_code))) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") + 
stat_compare_means(vjust = 1, size = 6.5) +
scale_fill_brewer(palette = "Paired") +
xlab("PFS Late Code") +
ylab("H3K4me2: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K4me2_signal_patient_PFS_late_code_k4DEgenes.pdf", width = 18, height = 18)

pt_clinical %>% ggplot(aes(x = log_pfs_days, y = value)) +
geom_point(aes(color = Analysis)) +
geom_smooth(aes(color = Analysis), method = "lm") +
theme_bw(base_size = 25) +
scale_color_brewer(palette = "Dark2") +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") +
xlab("Log PFS Days") +
ylab("H3K4me2: log2(Scaled Count + 1)")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K4me2_signal_patient_log_pfs_days.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% k4_TF_gene) %>% ggplot(aes(x = log_pfs_days, y = value)) +
geom_point(aes(color = log_pfs_days)) +
geom_smooth(color = "grey", method = "lm", se = FALSE) +
theme_bw(base_size = 25) +
scale_color_viridis() +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") +
xlab("Log PFS Days") +
ylab("H3K4me2: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/H3K4me2_signal_patient_log_pfs_days_k4DEgenes.pdf", width = 20, height = 10)




In [ ]:
## RNA
row_ind = which((rownames(rna_obj@assays$RNA@data) %>% gsub("COMMD3-BMI1", "COMMD3BMI1", .) %>% gsub(".*-", "", .) %>% toupper) %in% marker_gene)
rna_normMat = rna_obj@assays$RNA@data[row_ind, 1:10] %>% data.frame
colnames(rna_normMat) = colnames(rna_normMat) %>% gsub("Sample_", "", .) %>% gsub("_.*", "", .) %>% toupper
rownames(rna_normMat) = rownames(rna_normMat) %>% as.vector %>% as.character %>% gsub(".*-", "", .) %>% toupper
rna_normMat

In [ ]:

options(repr.plot.width=25, repr.plot.height=20)

pt_select_tmp = rna_normMat %>% data.frame
pt_select = pt_select_tmp %>% mutate(gene_num = rownames(pt_select_tmp)) %>% pivot_longer(cols = colnames(pt_select_tmp), names_to = "xnumber")
head(pt_select, 20)
pt_select %>% dim

pt_clinical = left_join(left_join(pt_select, clinical_data_select, by = "xnumber"), marker_gene_table[, c("Analysis", "GeneNameClean")], by = c("gene_num" = "GeneNameClean"))
pt_clinical %>% head

pt_clinical %>% ggplot(aes(x = as.factor(propscore_cd8_max), y = value)) +
geom_boxplot(aes(fill = Analysis)) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") + 
stat_compare_means(vjust = 1) +
scale_fill_brewer(palette = "Dark2") +
xlab("Propensity Score CD8 Max") +
ylab("RNA-seq: log2(Scaled Count + 1)")

ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/RNA_signal_patient_propscore_cd8_max.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% rna_TF_gene) %>% ggplot(aes(x = as.factor(propscore_cd8_max), y = value)) +
geom_boxplot(aes(fill = as.factor(propscore_cd8_max))) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") + 
stat_compare_means(vjust = 1, size = 6.5) +
scale_fill_brewer(palette = "Paired") +
xlab("Propensity Score CD8 Max") +
ylab("RNA-seq: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/RNA_signal_patient_propscore_cd8_max_rnaDEgenes.pdf", width = 18, height = 9)

pt_clinical %>% ggplot(aes(x = log_cd8_max, y = value)) +
geom_point(aes(color = Analysis)) +
geom_smooth(aes(color = Analysis), method = "lm") +
theme_bw(base_size = 25) +
scale_color_brewer(palette = "Dark2") +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") +
xlab("Log CD8 Max") +
ylab("RNA-seq: log2(Scaled Count + 1)")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/RNA_signal_patient_log_cd8_max.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% rna_TF_gene) %>% ggplot(aes(x = log_cd8_max, y = value)) +
geom_point(aes(color = log_cd8_max)) +
geom_smooth(color = "grey", method = "lm", se = FALSE) +
theme_bw(base_size = 25) +
scale_color_viridis() +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") +
xlab("Log CD8 Max") +
ylab("RNA-seq: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/RNA_signal_patient_log_cd8_max_rnaDEgenes.pdf", width = 20, height = 7)

## "pfscode",	"pfsdaysafterfirstcartcell", "pfs_late_code", "log_pfs_days"

options(repr.plot.width=25, repr.plot.height=20)

pt_clinical %>% ggplot(aes(x = as.factor(pfs_late_code), y = value)) +
geom_boxplot(aes(fill = Analysis)) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") + 
stat_compare_means(vjust = 1) +
scale_fill_brewer(palette = "Dark2") +
xlab("PFS Late Code") +
ylab("RNA-seq: log2(Scaled Count + 1)")

ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/RNA_signal_patient_PFS_late_code.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% rna_TF_gene) %>% ggplot(aes(x = as.factor(pfs_late_code), y = value)) +
geom_boxplot(aes(fill = as.factor(pfs_late_code))) +
theme_bw(base_size = 25) +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") + 
stat_compare_means(vjust = 1, size = 6.5) +
scale_fill_brewer(palette = "Paired") +
xlab("PFS Late Code") +
ylab("RNA-seq: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/RNA_signal_patient_PFS_late_code_rnaDEgenes.pdf", width = 18, height = 9)

pt_clinical %>% ggplot(aes(x = log_pfs_days, y = value)) +
geom_point(aes(color = Analysis)) +
geom_smooth(aes(color = Analysis), method = "lm") +
theme_bw(base_size = 25) +
scale_color_brewer(palette = "Dark2") +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 10, scales = "free") +
xlab("Log PFS Days") +
ylab("RNA-seq: log2(Scaled Count + 1)")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/RNA_signal_patient_log_pfs_days.pdf", width = 25, height = 18)

pt_clinical %>% filter(gene_num %in% rna_TF_gene) %>% ggplot(aes(x = log_pfs_days, y = value)) +
geom_point(aes(color = log_pfs_days)) +
geom_smooth(color = "grey", method = "lm", se = FALSE) +
theme_bw(base_size = 25) +
scale_color_viridis() +
facet_wrap(~factor(gene_num, levels = marker_gene), ncol = 5, scales = "free") +
xlab("Log PFS Days") +
ylab("RNA-seq: log2(Scaled Count + 1)") +
rremove("legend")
ggsave(filename = "CART_CUTRUN_Project/results/paper_figure/fig7/RNA_signal_patient_log_pfs_days_rnaDEgenes.pdf", width = 20, height = 7)
